In [1]:
import base64
import collections
import copy
import cProfile
import datetime
import gc
import itertools
import json
import math
import os
import operator
import pickle
import random
import re
import shutil
import sys
import time

import bokeh
import cv2
import hyperopt
from hyperopt import hp
import Image
import keras
from keras import *
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL as pil
import prophet
import pyflux
import pylab
import scipy
from scipy import signal
import seaborn as sns
import skimage
import sklearn
from sklearn import *
import statsmodels as sm
import tensorflow as tf
import tqdm

np.random.seed(1337)

%matplotlib inline

sns.set(font_scale=1.3)
mpl.rcParams['figure.figsize'] = 20, 12
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

linewidth = 1.0
dotsize = 15

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/lo

In [2]:
SPLIT_TEST_SIZE = 0.2

In [3]:
train_all_raw = pd.read_json('/media/ntfs/data/iceberg_classification/input/train.json')
train_all_size = len(train_all_raw)
print train_all_size

train_all_x = np.zeros((train_all_size, 75, 75, 3))
train_all_y = np.zeros(train_all_size)

for i in range(train_all_size):
    for channel in range(2):
        train_all_x[i, :, :, channel] = (np.asarray(train_all_raw.iloc[i, channel]).reshape(75, 75) + 50) / 50
    train_all_x[i, :, :, 2] = np.zeros((75, 75))
    train_all_y[i] = train_all_raw.iloc[i, 4]
    
del train_all_raw
gc.collect()

train_x, val_x, train_y, val_y = sklearn.model_selection.train_test_split(
                                    train_all_x,
                                    train_all_y,
                                    test_size=SPLIT_TEST_SIZE)

del train_all_x
del train_all_y
gc.collect()

val_x = val_x[:, :, :, 0:2]

train_size = len(train_x)
val_size = len(val_x)

print train_size
print val_size

1604
1283
321


In [4]:
def create_model(
        kernel_size,
    
        n_filters_input,
        activation_input,
        max_pooling_size_input,
        dropout_input,
    
        n_conv_layers,
        n_filters_conv,
        activation_conv,
        max_pooling_size_conv,
        dropout_conv,
    
        n_dense_layers,
        n_dense_neurons,
        activation_dense,
        dropout_dense,
    
        optimizer,
        loss):
    
    model = models.Sequential()

    # input layer
    model.add(layers.Conv2D(
                n_filters_input,
                (kernel_size, kernel_size),
                activation=activation_input,
                input_shape=(75, 75, 2)))
    model.add(layers.MaxPooling2D((max_pooling_size_input, max_pooling_size_input), dim_ordering='th'))
    model.add(keras.layers.Dropout(dropout_input))
    
    # conv layers
    for i in range(n_conv_layers):
        model.add(layers.Conv2D(
                    n_filters_conv,
                    (kernel_size, kernel_size),
                    activation=activation_conv))
        model.add(layers.MaxPooling2D((max_pooling_size_conv, max_pooling_size_conv), dim_ordering='th'))
        model.add(keras.layers.Dropout(dropout_conv))

    model.add(keras.layers.Flatten())
        
    # dense layers
    for i in range(n_dense_layers):
        model.add(keras.layers.Dense(n_dense_neurons, activation=activation_dense))
        model.add(keras.layers.Dropout(dropout_dense))
    
    #sigmoid layer
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=['accuracy'])
    
    return model

In [5]:
def create_augmented_data(
        rotation_range,
        width_shift_range,
        height_shift_range,
        shear_range,
        zoom_range,
        channel_shift_range,
        fill_mode,
        cval,
        horizontal_flip,
        vertical_flip,
        train_aug_size_multiplier):
    
    generator = keras.preprocessing.image.ImageDataGenerator(
                rotation_range=rotation_range,
                width_shift_range=width_shift_range,
                height_shift_range=height_shift_range,
                shear_range=shear_range,
                zoom_range=zoom_range,
                channel_shift_range=channel_shift_range,
                fill_mode=fill_mode,
                cval=cval,
                horizontal_flip=horizontal_flip,
                vertical_flip=vertical_flip)

    flow = generator.flow(train_x, train_y, batch_size=1)
    
    train_aug_size = int(train_size * (1 + train_aug_size_multiplier))

    train_aug_x = np.zeros((train_aug_size, 75, 75, 2))
    train_aug_y = np.zeros(train_aug_size)

    train_aug_x[:train_size, :, :, :] = train_x[:, :, :, 0:2]
    train_aug_y[:train_size] = train_y[:]

    i = train_size
    for f in flow:
        train_aug_x[i, :, :, 0:2] = f[0][0, :, :, 0:2]
        train_aug_y[i] = f[1]

        i += 1
        if i >= train_aug_size:
            break
            
    return train_aug_x[:, :, :, 0:2], train_aug_y

In [6]:
dry_run = False
trials = []

def evaluate_params(params):
    
    log_loss = 100.0
    try:
        #print params
        start_time = time.time()
        
        if not dry_run:
            model = create_model(
                        kernel_size=3,
    
                        n_filters_input=16,
                        activation_input='tanh',
                        max_pooling_size_input=2,
                        dropout_input=0.3,

                        n_conv_layers=2,
                        n_filters_conv=128,
                        activation_conv='relu',
                        max_pooling_size_conv=2,
                        dropout_conv=0.6,

                        n_dense_layers=2,
                        n_dense_neurons=128,
                        activation_dense='relu',
                        dropout_dense=0.3,

                        optimizer='rmsprop',
                        loss='binary_crossentropy')
            
            train_aug_x, train_aug_y = create_augmented_data(
                                            rotation_range=params['rotation_range'],
                                            width_shift_range=params['width_shift_range'],
                                            height_shift_range=params['height_shift_range'],
                                            shear_range=params['shear_range'],
                                            zoom_range=params['zoom_range'],
                                            channel_shift_range=params['channel_shift_range'],
                                            fill_mode=params['fill_mode'],
                                            cval=params['cval'],
                                            horizontal_flip=params['horizontal_flip'],
                                            vertical_flip=params['vertical_flip'],
                                            train_aug_size_multiplier=params['train_aug_size_multiplier'])

            history = model.fit(
                    x=train_aug_x,
                    y=train_aug_y,
                    epochs=60,
                    validation_data=(val_x, val_y),
                    batch_size=16,
                    verbose=0)

            val_acc = np.average(history.history['val_acc'][-10:])
            
            p = model.predict(val_x)
            log_loss = sklearn.metrics.log_loss(val_y, p)
        else:
            val_acc = 0.5
            log_loss = np.random.randn() + 100
            
        if np.isnan(log_loss):
            log_loss = 100.0

        trial = params.copy()
        trial['val_acc'] = val_acc
        trial['log_loss'] = log_loss
        trial['time'] = int(time.time() - start_time)
        trials.append(trial)
        
        is_best = ''
        if log_loss <= np.min([t['log_loss'] for t in trials]):
            is_best = '*'

        print 'trial={0:04d}     time={1:04}s     log_loss={2:2.5f}{3}'.format(
            len(trials),
            trial['time'],
            trial['log_loss'],
            is_best)
    except Exception as e:
        raise e
        pass
    
    return log_loss

In [7]:
space = {
    'rotation_range': 15 * hp.randint('rotation_range', 4),
    'width_shift_range': hp.randint('width_shift_range', 2) / 5.0,
    'height_shift_range': hp.randint('height_shift_range', 2) / 5.0,
    'shear_range': hp.randint('shear_range', 2) / 5.0,
    'zoom_range': hp.randint('zoom_range', 2) / 5.0,
    'channel_shift_range': hp.randint('channel_shift_range', 2) / 5.0,
    'fill_mode': hp.choice('fill_mode', ['constant', 'nearest']),
    'cval': hp.randint('cval', 1),
    'horizontal_flip': hp.choice('horizontal_flip', [True, False]),
    'vertical_flip': hp.choice('vertical_flip', [True, False]),
    'train_aug_size_multiplier': hp.choice('train_aug_size_multiplier', [1, 3])
}

In [8]:
trials = []
_ = hyperopt.fmin(evaluate_params,
    space=space,
    algo=hyperopt.tpe.suggest,
    max_evals=64)

trial=0001     time=0292s     log_loss=0.25785*
trial=0002     time=0289s     log_loss=0.38940
trial=0003     time=0148s     log_loss=100.00000
trial=0004     time=0148s     log_loss=0.35309
trial=0005     time=0288s     log_loss=0.45628
trial=0006     time=0288s     log_loss=0.31535
trial=0007     time=0147s     log_loss=100.00000
trial=0008     time=0149s     log_loss=0.42429
trial=0009     time=0149s     log_loss=0.31943
trial=0010     time=0290s     log_loss=0.37585
trial=0011     time=0290s     log_loss=0.29820
trial=0012     time=0289s     log_loss=0.25184*
trial=0013     time=0290s     log_loss=0.36319
trial=0014     time=0290s     log_loss=0.31643
trial=0015     time=0149s     log_loss=0.35185
trial=0016     time=0290s     log_loss=0.26698
trial=0017     time=0149s     log_loss=0.33346
trial=0018     time=0291s     log_loss=100.00000
trial=0019     time=0291s     log_loss=0.33479
trial=0020     time=0291s     log_loss=0.28401
trial=0021     time=0291s     log_loss=0.29127
trial

In [9]:
results = pd.DataFrame(trials).sort_values('log_loss')
results.to_csv('results.csv', index=False)

In [10]:
results.head(50).T

,27,34,45,22,55,43,11,0,53,37,...,4,41,48,52,44,63,36,31,58,33
channel_shift_range,0,0,0,0,0,0,0,0,0,0,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
cval,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fill_mode,constant,constant,constant,constant,constant,constant,constant,constant,constant,constant,...,nearest,nearest,nearest,constant,nearest,constant,constant,constant,nearest,constant
height_shift_range,0.2,0.2,0.2,0.2,0,0,0.2,0,0.2,0.2,...,0.2,0.2,0.2,0,0.2,0,0.2,0.2,0.2,0.2
horizontal_flip,True,True,False,True,False,True,True,True,True,False,...,False,True,False,False,True,True,True,True,True,True
log_loss,0.233352,0.237089,0.237484,0.238651,0.240087,0.245073,0.25184,0.257852,0.264249,0.265775,...,0.456281,0.457169,0.46228,0.477031,0.479726,0.484384,0.48771,0.488972,0.502607,0.546657
rotation_range,0,0,15,0,15,0,0,15,30,30,...,30,15,45,45,45,0,0,0,45,30
shear_range,0,0,0.2,0,0,0,0,0.2,0,0,...,0.2,0,0,0.2,0,0,0,0,0,0
time,291,292,295,291,153,152,289,292,298,151,...,288,293,297,297,295,300,293,291,297,293
train_aug_size_multiplier,3,3,3,3,1,1,3,3,3,1,...,3,3,3,3,3,3,3,3,3,3


In [11]:
results.describe().T

,count,mean,std,min,25%,50%,75%,max
channel_shift_range,64.0,0.053125,0.089031,0.000000,0.000000,0.000000,0.200000,0.200000
cval,64.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
height_shift_range,64.0,0.140625,0.092099,0.000000,0.000000,0.200000,0.200000,0.200000
log_loss,64.0,22.143432,41.523571,0.233352,0.290192,0.356643,0.492381,100.000000
rotation_range,64.0,15.234375,17.421700,0.000000,0.000000,15.000000,30.000000,45.000000
shear_range,64.0,0.059375,0.092099,0.000000,0.000000,0.000000,0.200000,0.200000
time,64.0,257.421875,62.295712,147.000000,254.500000,291.000000,295.000000,300.000000
train_aug_size_multiplier,64.0,2.500000,0.872872,1.000000,2.500000,3.000000,3.000000,3.000000
val_acc,64.0,0.864802,0.043514,0.746417,0.862072,0.883801,0.890810,0.915888
width_shift_range,64.0,0.137500,0.093435,0.000000,0.000000,0.200000,0.200000,0.200000
